In [46]:
import pandas as pd
from Bio.Seq import Seq

In [47]:
        df = pd.read_csv("/SGRNJ06/randd/USER/cjj/celedev/trust4/20220505/M_0417Bcell_14G_B3Nlib/03.assemble/assemble/M_0417Bcell_14G_B3Nlib_contig.csv", sep='\t', header=None)
        df.columns = ['barcode', 'is_cell', 'contig_id', 'high_confidence', 'length', 'chain', 'v_gene', 'd_gene', 'j_gene', 'c_gene', 'full_length', 'productive', 'cdr3', 'cdr3_nt', 'reads', 'umis']
        df['d_gene'] = df['d_gene'].apply(lambda x: x.split('(')[0] if not x == '*' else 'None')
        df['c_gene'] = df['c_gene'].apply(lambda x: x.split('(')[0] if not x == '*' else 'None')
        df['cdr3'] = df['cdr3_nt'].apply(lambda x: 'None' if "*" in str(Seq(x).translate()) or not len(x)%3==0 else str(Seq(x).translate()))
        df['productive'] = df['cdr3'].apply(lambda x: False if x=='None' else True)

/Personal/chenjunjie/.local/lib/python3.7/site-packages/Bio/Seq.py:2983: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  BiopythonWarning,


In [ ]:
/SGRNJ06/randd/USER/cjj/celedev/trust4/20220505/M_0417Bcell_14G_B3Nlib/03.assemble/assemble

In [38]:
filterbc_rep = pd.read_csv('/SGRNJ06/randd/USER/cjj/celedev/trust4/20220505/M_0417Bcell_14G_B3Nlib/03.assemble/assemble/barcoderepfl.tsv',sep='\t')
trust_rep = pd.read_csv('/SGRNJ06/randd/USER/cjj/celedev/trust4/20220505/M_0417Bcell_14G_B3Nlib/03.assemble/assemble/trust_filter_report.out', sep='\t')

In [39]:
            df.sort_values(by='umis', ascending=False, inplace=True)
            df_chain_heavy = df[df['chain']=='IGH']
            df_chain_light = df[(df['chain']=='IGK') | (df['chain']=='IGL')]
            df_chain_heavy = df_chain_heavy.drop_duplicates(['barcode'])
            df_chain_light = df_chain_light.drop_duplicates(['barcode'])
            df_for_clono = pd.concat([df_chain_heavy, df_chain_light], ignore_index=True)

In [31]:
        filterbc_rep = filterbc_rep.rename(columns = {'#barcode':'barcode'})
        filterbc = set(filterbc_rep['barcode'].tolist())
        trust_rep = trust_rep[trust_rep['cid_full_length'] >= 1]
        trust_rep = trust_rep[trust_rep['CDR3aa'] != 'out_of_frame']
        trust_rep = trust_rep.rename(columns = {'cid':'barcode', '#count':'count'})
        trust_rep['barcode'] = trust_rep['barcode'].apply(lambda x:x.split('_')[0])
        cdr3_list = list(trust_rep['CDR3aa'])
        cdr3_list = [i for i in cdr3_list if i.startswith('C')]
        cdr3_list = [i for i in cdr3_list if len(i)>=5]
        cdr3_list = [i for i in cdr3_list if 'UAG' or 'UAA' or 'UGA' not in i]
        trust_rep = trust_rep[trust_rep['CDR3aa'].isin(cdr3_list)]

In [32]:
        trust_rep_bc = set(trust_rep['barcode'].tolist())

        # df_for_clono = df_for_clono[df_for_clono['umis']>=3]
        # df_for_clono = df_for_clono[df_for_clono['reads']>=2]
        df_for_clono = df_for_clono[df_for_clono['barcode'].isin(filterbc)]
        df_for_clono = df_for_clono[df_for_clono['barcode'].isin(trust_rep_bc)]

In [35]:
len(filterbc)

3797

In [36]:
len(trust_rep_bc)

In [44]:
trust_rep

,#count,frequency,CDR3nt,CDR3aa,V,D,J,C,cid,cid_full_length
0,1723079,0.959271,TGTCAGCAGCGTAGCAACTGGCCTCCGACGTTC,CQQRSNWPPTF,IGKV3-11*01,.,IGKJ1*01,IGKC,TGATACGTTCAGGAGGTTGTTCCA_0,1
1,17297,0.018947,TGTGCGAGAGGGGCTGACTACTGG,CARGADYW,IGHV3-48*03,IGHD2-21*02,IGHJ4*02,.,TGATACGTTCAGGAGGTTGTTCCA_261,0
2,16480,0.018052,TGTGCGAGGGGTATAACTGGAAATGACTACTGG,CARGITGNDYW,IGHV3-20*01,IGHD1-20*01,IGHJ4*02,.,TTGGAGGTGTTGTCGGTGAACTGG_715,0
3,8261,0.004599,TGTCAGCAGCGTAGCAACTGGCCTCCGTCGTTC,CQQRSNWPPSF,IGKV3-11*01,.,IGKJ1*01,IGKC,TCAGGAGGGAGGATGGTACTAGTC_57736,0
4,7947,0.008705,TGTGCAAGAGAAGGGGAAATAGTAGTGGTTATTTACTTTGACTACTGG,CAREGEIVVVIYFDYW,IGHV6-1*01,IGHD3-22*01,IGHJ4*02,.,ATCACGTTTTACTCGCTTCGCACC_1499,0
...,...,...,...,...,...,...,...,...,...,...
120896,1,0.000001,TGTGCGAGGGGGTATAGCTGTGGCTGGCATTACTATGACTACTGG,CARGYSCGWHYYDYW,IGHV3-64*04,IGHD6-19*01,IGHJ4*02,.,TGGCTCAGTTGCGTACGACCTTAG_246697,0
120897,1,0.000001,TGTGCTAGTGGGGGGTATTTCTGGAACTCATGGTTCTTCGATCTCTGG,CASGGYFWNSWFFDLW,IGHV4-34*01,IGHD1-7*01,IGHJ2*01,.,TCATTGAGGAATCTGTACTTGATG_355926,0
120898,1,0.000001,TGTGCGAAAGAGGAAGTAGCAGGTGTCTACTGGTTCTTCGATCTCTGT,CAKEEVAGVYWFFDLC,IGHV3-23*01,IGHD2-2*01,IGHJ2*01,.,TCTCGGTTGACCTTAGGTACATCT_329566,0
120899,1,0.000001,TGTGCGAATGGGTGTATAGCAGTGGCTGGTCCCTTTGACTACTGG,CANGCIAVAGPFDYW,IGHV3-23*01,IGHD6-19*01,IGHJ4*02,.,GAGGATGGTCGAGCGTTTGGTATG_243410,0


In [45]:
trust_rep['cid_full_length'].value_counts()

0    120882
1        19
Name: cid_full_length, dtype: int64

In [49]:
df[df['chain']=='IGH']

,barcode,is_cell,contig_id,high_confidence,length,chain,v_gene,d_gene,j_gene,c_gene,full_length,productive,cdr3,cdr3_nt,reads,umis
4600,GGATTAGGTGCGATCTGTACATCT,True,GGATTAGGTGCGATCTGTACATCT_18189,True,569,IGH,IGHV4-39*06,IGHD4-11*01,IGHJ6*02,IGHE,True,True,CARDQGDYPYYYYYGMDVW,TGTGCGAGAGATCAGGGTGACTACCCTTACTACTACTACTACGGTA...,20,18
4601,TCGAGCGTTGTTCTCCTCTACGAC,True,TCGAGCGTTGTTCTCCTCTACGAC_37759,True,458,IGH,IGHV4-59*12,IGHD3-22*01,IGHJ6*02,IGHE,True,True,CARALPLYYYYYGMDVW,TGTGCGAGAGCCCTACCCCTTTACTACTACTACTACGGTATGGACG...,5,5
6037,TTGGTATGCAGATCTGTCCGTCTT,True,TTGGTATGCAGATCTGTCCGTCTT_1898,True,604,IGH,IGHV3-20*01,IGHD1-20*01,IGHJ4*02,IGHE,True,True,CARGITGIDYW,TGTGCGAGGGGTATAACTGGAATTGACTACTGG,60,50
6238,TACCACCACGATGTTTTCCAGTCG,True,TACCACCACGATGTTTTCCAGTCG_4280,True,638,IGH,IGHV1-2*04,IGHD3-22*01,IGHJ6*02,IGHE,True,True,CARVGDSSGYPYYYYYGMDVW,TGTGCGAGAGTCGGGGATAGTAGTGGTTACCCCTACTACTACTACT...,12,11
6244,GAGCCAATGGTCGTGTTCCAGTCG,True,GAGCCAATGGTCGTGTTCCAGTCG_18941,True,627,IGH,IGHV3-64*04,IGHD6-19*01,IGHJ3*02,IGHE,True,True,CARALYSSGWNDAFDIW,TGTGCGAGAGCCCTGTATAGCAGTGGCTGGAATGATGCTTTTGATA...,37,16


In [35]:
import pysam

In [36]:
handle_file = '/SGRNJ03/randd/cjj/Celescope/dev/CeleScope/celescope/tools/trust4/database/other/bcrtcr.fa'


In [37]:
handle_file = open(handle_file,'a+')
with pysam.FastxFile('/SGRNJ03/randd/cjj/Celescope/dev/CeleScope/celescope/tools/trust4/database/GRCm38/bcrtcr.fa') as f:
    for read in f:
        if 'IGHA' in str(read.name):
            handle_file.write(str(read) + '\n')
        if 'IGHM' in str(read.name):
            handle_file.write(str(read) + '\n')
        if 'IGHG' in str(read.name):
            handle_file.write(str(read) + '\n')
        if 'IGHD' in str(read.name):
            handle_file.write(str(read) + '\n')
        if 'IGHE' in str(read.name):
            handle_file.write(str(read) + '\n')
        if 'IGKC' in str(read.name):
            handle_file.write(str(read) + '\n')
        if 'IGLC' in str(read.name):
            handle_file.write(str(read) + '\n')
handle_file.close()

In [38]:
handle_file = '/SGRNJ03/randd/cjj/Celescope/dev/CeleScope/celescope/tools/trust4/database/other/IGH.fa'
handle_file = open(handle_file,'a+')
with pysam.FastxFile('/SGRNJ03/randd/cjj/Celescope/dev/CeleScope/celescope/tools/trust4/database/GRCm38/IGH.fa') as f:
    for read in f:
        if 'IGHA' in str(read.name):
            handle_file.write(str(read) + '\n')
        if 'IGHM' in str(read.name):
            handle_file.write(str(read) + '\n')
        if 'IGHG' in str(read.name):
            handle_file.write(str(read) + '\n')
        if 'IGHD' in str(read.name):
            handle_file.write(str(read) + '\n')
        if 'IGHE' in str(read.name):
            handle_file.write(str(read) + '\n')
handle_file.close()

In [39]:
handle_file = '/SGRNJ03/randd/cjj/Celescope/dev/CeleScope/celescope/tools/trust4/database/other/IGK.fa'
handle_file = open(handle_file,'a+')
with pysam.FastxFile('/SGRNJ03/randd/cjj/Celescope/dev/CeleScope/celescope/tools/trust4/database/GRCm38/IGK.fa') as f:
    for read in f:
        if 'IGKC' in str(read.name):
            handle_file.write(str(read) + '\n')
handle_file.close()

In [40]:
handle_file = '/SGRNJ03/randd/cjj/Celescope/dev/CeleScope/celescope/tools/trust4/database/other/IGL.fa'
handle_file = open(handle_file,'a+')
with pysam.FastxFile('/SGRNJ03/randd/cjj/Celescope/dev/CeleScope/celescope/tools/trust4/database/GRCm38/IGL.fa') as f:
    for read in f:
        if 'IGLC' in str(read.name):
            handle_file.write(str(read) + '\n')
handle_file.close()

In [44]:
handle_file = '/SGRNJ03/randd/cjj/Celescope/dev/CeleScope/celescope/tools/trust4/database/other/test.fa'
handle_file = open(handle_file,'a+')
with pysam.FastxFile('/SGRNJ03/randd/cjj/Celescope/dev/CeleScope/celescope/tools/trust4/database/GRCm38/IMGT+C.fa') as f:
    for read in f:
        if 'IGHA' in str(read.name):
            handle_file.write(str(read) + '\n')
        if 'IGHM' in str(read.name):
            handle_file.write(str(read) + '\n')
        if 'IGHG' in str(read.name):
            handle_file.write(str(read) + '\n')
        if 'IGHD' in str(read.name):
            handle_file.write(str(read) + '\n')
        if 'IGHE' in str(read.name):
            handle_file.write(str(read) + '\n')
        if 'IGKC' in str(read.name):
            handle_file.write(str(read) + '\n')
        if 'IGLC' in str(read.name):
            handle_file.write(str(read) + '\n')
handle_file.close()

In [55]:
handle_file = '/SGRNJ03/randd/cjj/Celescope/dev/CeleScope/celescope/tools/trust4/database/other/test.fa'
handle_file = open(handle_file,'a+')
with pysam.FastxFile('/SGRNJ03/randd/cjj/Celescope/dev/CeleScope/celescope/tools/trust4/database/GRCm38/IMGT+C.fa') as f:
    for read in f:
        if 'IGHA' in str(read.name):
            length = len(read.sequence)
            handle_file.write(read.name + '\n')
            if length / 60 > 1:
                for i in range(length//60 + 1):
                    handle_file.write(read.sequence[i*60:(i+1)*60] + '\n')
        if 'IGHM' in str(read.name):
            length = len(read.sequence)
            handle_file.write(read.name + '\n')
            if length / 60 > 1:
                for i in range(length//60 + 1):
                    handle_file.write(read.sequence[i*60:(i+1)*60] + '\n')
        if 'IGHG' in str(read.name):
            length = len(read.sequence)
            handle_file.write(read.name + '\n')
            if length / 60 > 1:
                for i in range(length//60 + 1):
                    handle_file.write(read.sequence[i*60:(i+1)*60] + '\n')
        if 'IGHD' in str(read.name):
            length = len(read.sequence)
            handle_file.write(read.name + '\n')
            if length / 60 > 1:
                for i in range(length//60 + 1):
                    handle_file.write(read.sequence[i*60:(i+1)*60] + '\n')
        if 'IGHE' in str(read.name):
            length = len(read.sequence)
            handle_file.write(read.name + '\n')
            if length / 60 > 1:
                for i in range(length//60 + 1):
                    handle_file.write(read.sequence[i*60:(i+1)*60] + '\n')
        if 'IGKC' in str(read.name):
            length = len(read.sequence)
            handle_file.write(read.name + '\n')
            if length / 60 > 1:
                for i in range(length//60 + 1):
                    handle_file.write(read.sequence[i*60:(i+1)*60] + '\n')
        if 'IGLC' in str(read.name):
            length = len(read.sequence)
            handle_file.write(read.name + '\n')
            if length / 60 > 1:
                for i in range(length//60 + 1):
                    handle_file.write(read.sequence[i*60:(i+1)*60] + '\n')
handle_file.close()

In [ ]:

with pysam.FastxFile('/SGRNJ03/randd/cjj/Celescope/dev/CeleScope/celescope/tools/trust4/database/GRCm38/IMGT+C.fa') as f:
    for read in f:
        if 'IGHA' in str(read.name):
            print(read)
        if 'IGHM' in str(read.name):
            print(read)
        if 'IGHG' in str(read.name):
            print(read)
        if 'IGHD' in str(read.name):
            print(read)
        if 'IGHE' in str(read.name):
            print(read)
        if 'IGKC' in str(read.name):
            print(read)
        if 'IGLC' in str(read.name):
            print(read)

In [59]:

with open('/SGRNJ03/randd/cjj/Celescope/dev/CeleScope/celescope/tools/trust4/database/GRCm38/IMGT+C.fa') as f:
    for line in f:
        print(line.strip('\n'))


>IGHA*01
............GAGTCTGCGAGAAATCCCACCATCTACCCACTGACACTCCCACCAGTC
CTGTGC............AGTGATCCCGTGATAATCGGCTGCCTGATTCACGATTACTTC
CCTTTC...GGCACGATGAATGTGACCTGGGGAAAGAGTGGGAAGGATATA.........
...ACCACCGTGAACTTTCCACCTGCCCTCGCCTCTGGG..................GGA
CGGTACACCATGAGCAGCCAGTTAACCCTGCCAGCTGTCGAGTGC......CCAGAAGGA
GAGTCCGTGAAATGTTCCGTGCAACATGAC.........TCTAACCCCGTCCAAGAATTG
GATGTGAATTGCTCT
GGTCCTACTCCTCCTCCTCCTATTACTATTCCT..................TCCTGCCAG
CCCAGCCTGTCACTGCAGCGGCCAGCTCTTGAGGACCTGCTCCTG......GGTTCAGAT
GCCAGCATCACATGTACTCTGAATGGCCTGAGAAATCCT...GAGGGAGCTGCTTTCACC
TGGGAGCCCTCCACTGGGAAGGAT............GCAGTGCAGAAGAAAGCTGCGCAG
AATTCCTGCGGC.....................TGCTACAGTGTGTCCAGCGTCCTGCCT
GGCTGTGCTGAGCGCTGG......AACAGTGGCGCATCATTCAAGTGCACAGTTACCCAT
CCTGAG......TCTGGCACCTTAACTGGCACAATTGCCAAAGTCACA
............GTGAACACCTTCCCACCCCAGGTCCACCTGCTACCGCCGCCGTCGGAG
GAGCTGGCCCTG......AATGAGCTCTTGTCCCTGACATGCCTGGTGCGAGCTTTCAAC
CCT......AAAGAAGTGCTGGTGCGATGGCTGCATGGAAATGAGGAGCTGTCCCCAGAA
...AGCTACCT